<a href="https://colab.research.google.com/github/sillsphelyx/Geospatial-Analysis/blob/main/NDVI_Image_Plotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api
!earthengine authenticate

Authenticate: Limited support in Colab. Use ee.Authenticate() or --auth_mode=notebook instead.
Authenticate: Credentials already exist.  Use --force to refresh.


In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-sillsphelyx')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Insert the directoryimport sys
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/Geo-Spatial-Analysis')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install folium
!pip install rasterio

In [ ]:
import folium
import rasterio
from rasterio.mask import mask
import geopandas as gpd

In [ ]:
def shapefile_to_ee_geometry(shapefile_folder_path):
    gdf = gpd.read_file(shapefile_folder_path)

    # Validate geometry: Ensure each polygon has at least 3 points
    for index, row in gdf.iterrows():
        if len(row.geometry.exterior.coords) < 3:
            # Handle invalid geometries (e.g., remove or repair)
            print(f"Warning: Polygon at index {index} has less than 3 points. Skipping or fixing this geometry might be necessary.")
            gdf.drop(index, inplace=True)  # Example: drop the invalid polygon
            # Alternative: You could attempt to repair the geometry using shapely operations.

    # Proceed if you have valid geometries:
    if not gdf.empty:
        geojson = gdf.iloc[0].geometry.__geo_interface__
        return ee.Geometry(geojson)
    else:
        raise ValueError("The shapefile contains no valid polygons.")

In [ ]:
shapefile_folder_path = "/content/drive/MyDrive/Colab Notebooks/Geo-Spatial-Analysis/Felix_shape_files/Felix_shape.shp"
roi = shapefile_to_ee_geometry(shapefile_folder_path)

In [ ]:
import ee
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Define your Region of Interest (RoI)


# Load an Earth Engine Image Collection (e.g., Sentinel-2)
collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
    .filterBounds(roi) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Select the least cloudy image
image = collection.sort('CLOUDY_PIXEL_PERCENTAGE').first()

# Calculate NDVI
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')  # B8 is NIR, B4 is RED

# Clip the NDVI image to the RoI
ndvi_clipped = ndvi.clip(roi)

# Export NDVI to Google Drive
task = ee.batch.Export.image.toDrive(
    image=ndvi_clipped,
    description='NDVI_Image',
    scale=10,  # Resolution in meters
    region=roi.getInfo()['coordinates'],
    fileFormat='GeoTIFF',
    maxPixels=1e13
)
task.start()

# Get pixel-level NDVI values
samples = ndvi_clipped.sample(
    region=roi,
    scale=10,  # Match the image resolution
    numPixels=100,  # Number of pixels to sample
    geometries=True
).getInfo()

# Print sample pixel NDVI values
#for feature in samples['features']:
#    coords = feature['geometry']['coordinates']
 #  ndvi_value = feature['properties']['NDVI']
 #   print(f'Coordinates: {coords}, NDVI: {ndvi_value}')

# Visualize NDVI using geemap
Map = geemap.Map()
Map.centerObject(roi, 10)
# Add Google Earth imagery as a basemap
Map.add_basemap('SATELLITE')  # Use 'SATELLITE' for Google Earth imagery
# Define NDVI visualization parameters with thicker green for higher values
ndvi_vis = {
    'min': 0,  # Minimum NDVI value (e.g., 0 for no vegetation)
    'max': 1,  # Maximum NDVI value (e.g., 1 for dense vegetation)
    'palette': ['blue', 'yellow', 'green', 'green']  # Add shades for high NDVI
}
Map.addLayer(ndvi_clipped, ndvi_vis, 'NDVI')
Map.addLayer(roi, {}, 'ROI')
Map


Map(center=[-10.21210781720941, 31.26423237504809], controls=(WidgetControl(options=['position', 'transparent_…